# imports

In [1]:
import requests
import json
import sys
import io
import logging
import ipyleaflet as leaflet
import ipywidgets as widgets

# settings

In [2]:
st_url = "http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0"
where = "/Things"

querypolygon = (
    {"lat": 48.530143, "lon": 10.686774},
    {"lat": 48.530143, "lon": 11.162049},
    {"lat": 48.231226, "lon": 11.162049},
    {"lat": 48.231226, "lon": 10.686774},
    {"lat": 48.530143, "lon": 10.686774},
)

CAZ = (
    {"lon": 10.868776048246, "lat": 48.3329961062607},
    {"lon": 10.8707511122279, "lat": 48.3869470549635},
    {"lon": 10.924748760677, "lat": 48.3860560744848},
    {"lon": 10.9227167726086, "lat": 48.3321068025163},
    {"lon": 10.868776048246, "lat": 48.3329961062607}
)

starttime = "1018-08-28T10:00:00.001Z"
endtime = "2018-12-31T00:00:00.001Z"

mapwidth = "950px"
mapheight = "1400px"

# logging

In [3]:
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(asctime)s - %(levelname)s(%(name)s): %(message)s")

fh = logging.FileHandler('./log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)

sh = logging.StreamHandler(sys.stderr)
sh.setLevel(logging.ERROR)
sh.setFormatter(formatter)
log.addHandler(sh)

# get request

In [4]:
def GETRequest(url, qdata=None):
        """TODO: add docstring"""
        
        if qdata == None:
            qdata = ""
        else:
            qdata = "?" + qdata
        
        p = requests.get(url + qdata)
        body = b"";
        for chunk in p.iter_content(chunk_size=128):
            body += chunk
        body = body.decode("utf8")
        body = body.replace("\n", " ").replace("\r", "")
        if (p.status_code  == 200):
            log.debug("entities retrieved: " + url + qdata)
            return body
        else:
            log.error("cannot get entities: " + url + qdata + \
                      " - exit code: " + str(p.status_code) + " - body: " + body)
            raise TypeError("invalid request")

# make querry

In [5]:
polygonstring = ""
for elem in querypolygon:
    polygonstring += str(elem["lon"]) + " " + str(elem["lat"]) + ","

qdata = (
    "$expand=Locations" +
    "&$filter=" +
    
    "st_within(Locations/location, geography'POLYGON ((" +
    polygonstring[:-1] +
    "))')" +
    
    " and " +
    "HistoricalLocations/time ge " + starttime +
    " and " +
    "HistoricalLocations/time le " + endtime
)

# get entities

In [6]:
def GetEntities(st_url, where, qdata=None):
    entities = list()

    retval = json.loads(GETRequest(st_url + where, qdata))
    entities.extend(retval["value"])

    allkeys = [*retval]
    while "@iot.nextLink" in allkeys:
        nexturl = retval["@iot.nextLink"]
        retval = json.loads(GETRequest(nexturl))
        entities.extend(retval["value"])
        allkeys = [*retval]
        
    return entities

# create map

In [7]:
m = leaflet.Map(
    basemap = leaflet.basemaps.Esri.WorldStreetMap,
    center = (48.354912, 10.902233),
    zoom = 14,
    close_popup_on_click = False,
    layout=widgets.Layout(width=mapwidth, height=mapheight)
)

# station markers
entities = GetEntities(st_url, where, qdata)
for elem in entities:
    coords = elem["Locations"][0]["location"]["coordinates"]
    coords.reverse()
    
    name = elem["name"]
    description = elem["description"]
    iotid = elem["@iot.id"]
    try:
        properties = json.dumps(elem["properties"])
    except KeyError:
        properties = "None"
    message = widgets.HTML();
    message.value = (
        "@iot.id: " + iotid +  "<br>" +
        "name: " + name +  "<br>" +
        "description: " + description +  "<br>" +
        "coordinates: " + str(coords) + "<br>" +
        "properties: " + properties
    )
    
    marker = leaflet.Marker(location = coords, draggable = False)
    marker.popup = message
    
    m.add_layer(marker)
    
# CAZ polygon
polygon = leaflet.Polygon(
    locations = [[elem["lat"], elem["lon"]] for elem in CAZ],
    color = "green",
    fill_color = "green"
)
m.add_layer(polygon);

message = widgets.HTML()
message.value = "CAZ"
popup = leaflet.Popup(
    location = (CAZ[2]["lat"], CAZ[2]["lon"]),
    child = message,
    close_button = False,
    auto_close = False,
    close_on_escape_key = False
)
m.add_layer(popup)

# save map

In [8]:
from ipywidgets.embed import embed_minimal_html
embed_minimal_html('map.html', views=[m], title="MapDemo")

In [9]:
m

Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{…